# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

In [0]:
df1 = spark.read.option("delimiter", "\t").csv('dbfs:/FileStore/shared_uploads/david.rubiochavida@tajamar365.com/airbnb_last_review.tsv', header=True, inferSchema=True)

df2 = spark.read.csv('dbfs:/FileStore/shared_uploads/david.rubiochavida@tajamar365.com/airbnb_price.csv', header=True, inferSchema=True)

df3 = spark.read.option("header", "true").csv('dbfs:/FileStore/shared_uploads/david.rubiochavida@tajamar365.com/airbnb_room_type.csv')

df1.show()

df2.show()

df3.show()


+----------+----------------+---------------+
|listing_id|       host_name|    last_review|
+----------+----------------+---------------+
|      2595|        Jennifer|    May 21 2019|
|      3831|     LisaRoxanne|   July 05 2019|
|      5099|           Chris|   June 22 2019|
|      5178|        Shunichi|   June 24 2019|
|      5238|             Ben|   June 09 2019|
|      5295|            Lena|   June 22 2019|
|      5441|            Kate|   June 23 2019|
|      5803|          Laurie|   June 24 2019|
|      6021|         Claudio|   July 05 2019|
|      6848|   Allen & Irina|   June 29 2019|
|      7097|            Jane|   June 28 2019|
|      7322|            Doti|   July 01 2019|
|      7726|Adam And Charity|   June 22 2019|
|      8024|           Lisel|   July 01 2019|
|      8025|           Lisel|January 01 2019|
|      8110|           Lisel|   July 02 2019|
|      8490|        Nathalie|   June 19 2019|
|      8505|         Gregory|   June 23 2019|
|      9518|            Shon|   Ju

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.

In [0]:
from pyspark.sql.functions import to_date, col

# Pasamos la fecha a un formato legible
df1 = df1.withColumn("last_review_date", to_date(col("last_review"), "MMMM dd yyyy"))

# Obtener la primera y última fecha de reseña
first_review_date = df1.orderBy("last_review_date").first()["last_review_date"]
last_review_date = df1.orderBy(col("last_review_date").desc()).first()["last_review_date"]

# Imprimir las fechas
print("Fecha de la primera reseña:", first_review_date)
print("Fecha de la última reseña:", last_review_date)



Fecha de la primera reseña: 2019-01-01
Fecha de la última reseña: 2019-07-09


- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.

In [0]:
# Filtramos las filas donde el tipo de habitacion es privada y contamos 
private_rooms_count = df3.filter(df3.room_type == 'private room').count()

# Mostramos el resultado
private_rooms_count


Out[17]: 392

- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.

In [0]:
from pyspark.sql.functions import col, regexp_replace

# limpiamos el dato para eliminar la palabra dollar y que los registros se guarden como float
df2_cleaned = df2.withColumn("precio_limpio", regexp_replace(col("price"), " dollars", "").cast("float"))

# Calcular el precio promedio y redondearlo a dos decimales
average_price = df2_cleaned.agg({"precio_limpio": "avg"}).collect()[0][0]

# Redondear a dos decimales
average_price_rounded = round(average_price, 2)

# Guardar el resultado en una variable
average_price_rounded


Out[21]: 141.78


- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
from pyspark.sql import Row

# crear una fila con los datos que queremos en cada sitio
review_dates_row = Row(first_review_date=first_review_date, 
                       last_review_date=last_review_date, 
                       private_rooms_count=private_rooms_count, 
                       average_price_rounded=average_price_rounded)

# Crear el DataFrame con una sola fila usando la Row
review_dates = spark.createDataFrame([review_dates_row])

# mostramos la respuesta
review_dates.show()


+-----------------+----------------+-------------------+---------------------+
|first_review_date|last_review_date|private_rooms_count|average_price_rounded|
+-----------------+----------------+-------------------+---------------------+
|       2019-01-01|      2019-07-09|                392|               141.78|
+-----------------+----------------+-------------------+---------------------+

